Idea: Since Kakuro puzzles can expressed as a system of equations of sums, can linear algebra be used to simultaneously solve all unknowns? This is both an exploration of ways to build a Kakuro puzzle solver in general, as well as a review of linear algebra and matrix math.

# The Puzzle
Specific Puzzle: Page 52 of "The Book of Kakuro"

|Puz|5\\|14\\|17\\|39\\|6\\|x|x|25\\|14\\|
|---|---|---|---|---|---|---|---|---|---|
|\\19|?|?|?|?|?|30\\|\\11|?|?|
|\22|?|?|?|?|?|?|15\\10|?|?|
|x|x|30\\42|?|?|?|?|?|?|?|
|x|7\\23|?|?|?|20\\8|?|?|?|x|
|\\33|?|?|?|?|?|?|?|16\\|7\\|
|\\12|?|?|\\23|?|?|?|?|?|?|
|\\7|?|?|x|\\35|?|?|?|?|?|

14 down equations

12 across equations

50 unknowns

will need to create a 26x50 matrix of unknowns to solve at once. (Need to generate 24 more equations somewhere to make matrix square and invertible)

## Converted to variables

|Puz|d1\\|d2\\|d3\\|d4\\|d5\\|x|x|d6\\|d7\\|
|---|---|---|---|---|---|---|---|---|---|
|\\a1|1|2|3|4|5|d8\\|\\a2|6|7|
|\a3|8|9|10|11|12|13|d9\\a4|14|15|
|x|x|d10\\a5|16|17|18|19|20|21|22|
|x|d11\\a6|23|24|25|d12\\a7|26|27|28|x|
|\\a8|29|30|31|32|33|34|35|d13\\|d14\\|
|\\a9|36|37|\\a10|38|39|40|41|42|43|
|\\a11|44|45|x|\\a12|46|47|48|49|50|

In [1]:
import numpy as np

Structure "sums" as a vertical array [d_1,d_2,...,d_n,a_1,a_2,...,a_m]

In [2]:
sums = np.asmatrix([5,14,17,39,6,25,14,30,15,30,7,20,16,7,19,11,22,10,42,23,8,33,12,23,7,35]).transpose()
relations = np.zeros([26,50])

In [3]:
equations = {
    'd1': [1,8],
    'd2': [2,9],
    'd3': [3,10,16,24,31],
    'd4': [4,11,17,25,32,38],
    'd5': [5,12,18],
    'd6': [6,14,21,28],
    'd7': [7,15,22],
    'd8': [13,19,26,34,40,47],
    'd9': [20,27,35,41,48],
    'd10': [23,30,37,45],
    'd11': [29,36,44],
    'd12': [33,39,46],
    'd13': [42,49],
    'd14': [43,50],
    'a1': [1,2,3,4,5],
    'a2': [6,7],
    'a3': [8,9,10,11,12,13],
    'a4': [14,15],
    'a5': [16,17,18,19,20,21,22],
    'a6': [23,24,25],
    'a7': [26,27,28],
    'a8': [29,30,31,32,33,34,35],
    'a9': [36,37],
    'a10': [38,39,40,41,42,43],
    'a11': [44,45],
    'a12': [46,47,48,49,50],
}

A quick checksum for entry into equations. Each variable should be used twice -> for 50 unknowns I should have 100 entries across all equations.

In [8]:
sum([len(x) for x in equations.values()])

100

In [4]:
for eq, (key, unknowns) in enumerate(equations.items()):
#     print(eq,key,unknowns)
    for unknown in unknowns:
        relations[eq,unknown-1] = 1 #correct for 1-index of unknowns to 0 index convention

Problem: with 26 equations and 50 unknowns, there aren't enough equations to simultaneously solve using a inverse matrix of relations. Will either need to generate 24 more equations somehow leveraging the properties of non-repeating digits or will need to change method.

In [7]:
np.linalg.matrix_rank(relations)

25

Rank of the relations dictates how many linearly independent equations are defined by the puzzle. If the rank is 25 despite 26 input equations, one is a linear combination of other equations. Conveniently, 25 is half of the unknowns... can this info be used somehow to transform relations to a square matrix?

Every equation in Kakuro has a "complementary" equation, using the remaining of the digits [1-9] adding up to 45-sum(original equation). If added into set of equations, this should provide another 26 equations (assuming no 45 sums are in the puzzle)

In [11]:
info = {
    'd1': {
        'unknowns': [1,8],
        'total': 5,
        'digits': 2,
          },
    'd2': {
        'unknowns': [2,9],
        'total': 14,
        'digits': 2,
          },
    'd3': {
        'unknowns': [3,10,16,24,31],
        'total': 17,
        'digits': 5,
          },
    'd4': {
        'unknowns': [4,11,17,25,32,38],
        'total': 39,
        'digits': 6,
          },
    'd5': {
        'unknowns': [5,12,18],
        'total': 6,
        'digits': 3,
          },
    'd6': {
        'unknowns': [6,14,21,28],
        'total': 25,
        'digits': 4,
          },
    'd7': {
        'unknowns': [7,15,22],
        'total': 14,
        'digits': 3,
          },
    'd8': {
        'unknowns': [13,19,26,34,40,47],
        'total': 30,
        'digits': 6,
          },
    'd9': {
        'unknowns': [20,27,35,41,48],
        'total': 15,
        'digits': 5,
          },
    'd10': {
        'unknowns': [23,30,37,45],
        'total': 30,
        'digits': 4,
          },
    'd11': {
        'unknowns': [29,36,44],
        'total': 7,
        'digits': 3,
          },
    'd12': {
        'unknowns': [33,39,46],
        'total': 20,
        'digits': 3,
          },
    'd13': {
        'unknowns': [42,49],
        'total': 16,
        'digits': 2,
          },
    'd14': {
        'unknowns': [43,50],
        'total': 7,
        'digits': 2,
          },
    'a1': {
        'unknowns': [1,2,3,4,5],
        'total': 19,
        'digits': 5,
          },
    'a2': {
        'unknowns': [6,7],
        'total': 11,
        'digits': 2,
          },
    'a3': {
        'unknowns': [8,9,10,11,12,13],
        'total': 22,
        'digits': 6,
          },
    'a4': {
        'unknowns': [14,15],
        'total': 10,
        'digits': 2,
          },
    'a5': {
        'unknowns': [16,17,18,19,20,21,22],
        'total': 42,
        'digits': 7,
          },
    'a6': {
        'unknowns': [23,24,25],
        'total': 23,
        'digits': 3,
          },
    'a7': {
        'unknowns': [26,27,28],
        'total': 8,
        'digits': 3,
          },
    'a8': {
        'unknowns': [29,30,31,32,33,34,35],
        'total': 33,
        'digits': 7,
          },
    'a9': {
        'unknowns': [36,37],
        'total': 12,
        'digits': 2,
          },
    'a10': {
        'unknowns': [38,39,40,41,42,43],
        'total': 23,
        'digits': 6,
          },
    'a11': {
        'unknowns': [44,45],
        'total': 7,
        'digits': 2,
          },
    'a12': {
        'unknowns': [46,47,48,49,50],
        'total': 35,
        'digits': 5,
          },
}